In [1]:
import numpy as np
import open3d as o3d

In [2]:
def getCenterPoint(points_list):
    count = len(points_list)
    #init
    center = np.zeros(3)
    for i in range(count):
        center[0] += points_list[i][0]
        center[1] += points_list[i][1]
        center[2] += points_list[i][2]
    return center/count

In [3]:
def displayPoints(points_list):
    temp = o3d.geometry.PointCloud()
    temp.points = o3d.utility.Vector3dVector(np.asarray(points_list))
    o3d.visualization.draw_geometries([temp])

In [4]:
def duplicateSurface(numny_array_points):
    temp = []
    for i in numny_array_points:
        temp.append([i[0],i[1],i[2]+0.1])
    return np.asarray(temp)

In [6]:
pcd = o3d.io.read_point_cloud("./clean points/dwnsample_pose_1.ply")

In [7]:
center_point = getCenterPoint(np.asarray(pcd.points))
center_point[2]

2.3189816079884578

In [8]:
#downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
pcd.estimate_normals()
pcd.orient_normals_towards_camera_location(camera_location=np.asarray([center_point[0], center_point[1], 0.]))
#o3d.visualization.draw_geometries([pcd])

True

In [11]:
temp = np.asarray(pcd.points)
new_points = duplicateSurface(temp)
new_pcl = o3d.geometry.PointCloud()
new_pcl.points = o3d.utility.Vector3dVector(np.concatenate((temp,new_points)))
new_pcl.normals = o3d.utility.Vector3dVector(np.concatenate((np.asarray(pcd.normals),-np.asarray(pcd.normals))))
new_pcl.paint_uniform_color([0.8, 0.8, 0.8])
#o3d.visualization.draw_geometries([new_pcl])
#o3d.io.write_point_cloud("./clean points/dupback_pose_1.ply", new_pcl)

geometry::PointCloud with 1122 points.

In [12]:
print('run Poisson surface reconstruction')
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(new_pcl, depth=8)
print(mesh)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

run Poisson surface reconstruction
geometry::TriangleMesh with 3617 points and 7050 triangles.


In [10]:
# pcd.estimate_normals()
# distances = pcd.compute_nearest_neighbor_distance()
# avg_dist = np.mean(distances)
# radius = 4*avg_dist
# mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     pcd,
#     o3d.utility.DoubleVector([radius,radius*2]))
# o3d.visualization.draw_geometries([mesh])

In [11]:
o3d.io.write_triangle_mesh('./Mesh/Poisson Mesh/pose_1.ply',mesh)

True